In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('df.csv', index_col= 0)

In [ ]:
df.sample(5)

## ¿Cómo se distribuye la cantidad de vuelos reservados por mes durante el año?

In [ ]:
df_reservas_mensuales = df.groupby('month')['flights_booked'].sum().reset_index()

In [ ]:
sns.barplot(
    data=df_reservas_mensuales,
    x='month',
    y='flights_booked');

In [ ]:
sns.lineplot(
    data=df_reservas_mensuales,
    x='month',
    y='flights_booked'
);

Vemos que hay una subida significativa de reservas entre abril y julio. A partir de este mes bajan, para luego volver a subir entre noviembre y diciembre, seguramente por fechas navideñas

## ¿Existe una relación entre la distancia de los vuelos y los puntos acumulados por los cliente?

In [ ]:
sns.scatterplot(
    data=df,
    x='distance',
    y='points_accumulated',
    hue= 'loyalty_card');

vemos que claramente hay una relación clara entre la distancia volada y los puntos acumulados, siendo que cuantas mas millas/km se recorren, más puntos se acumulan. Las distintas líneas que vemos se deben a que la compañia utiliza distintos niveles de membresía, porque, si se tiene un nivel de membresía mas bajo, aunque se vuele más, los puntos van a ser menos que los de alguien con un nivel de membresía mas alto

## ¿Cuál es la distribución de los clientes por provincia o estado?

In [ ]:
df_distribucion = df['province'].value_counts().reset_index() # .value_counts() cuenta la frecuencia y .reset_index() la prepara para Seaborn
df_distribucion.columns = ['province', 'customer_count'] # Nombrar las columnas

sns.barplot(
    data=df_distribucion,
    x='customer_count', 
    y='province');


Aqui vemos que el mayor numero de clientes se encuentran en Ontario, mientras que el menor se encuentra en PrinceEdward Island.

## ¿Cómo se compara el salario promedio entre los diferentes niveles educativos de los clientes?

In [ ]:
education = ['High School or Below', 'College', 'Bachelor', 'Master', 'Doctor'] #definimos las columnas que tienen que aparecer

In [ ]:
# Calcular el salario promedio por educación
df_salario_promedio = df.groupby('education')['salary'].mean().reindex(education).reset_index()

sns.barplot(
    data=df_salario_promedio,
    x='education',
    y='salary');

In [ ]:
#buscamos la varianza de sueldo por educacion
# 1. Calcular la Desviación Estándar (std) del salario por nivel educativo
df_std_salary = df.groupby('education')['salary'].std().sort_values(ascending=False).reset_index()

# 2. Calcular la Varianza (var) del salario por nivel educativo
df_var_salary = df.groupby('education')['salary'].var().sort_values(ascending=False).reset_index()

print("📈 Desviación Estándar (Std) del Salario por Nivel Educativo:")
display(df_std_salary)

print("\n")

print("📉 Varianza (Var) del Salario por Nivel Educativo:")
df_var_salary

In [ ]:
sns.boxplot(
    data=df.query("education == 'High School or Below'"),
    x='salary'
    )


In [ ]:
sns.boxplot(
    data=df.query("education == 'College'"),
    x='salary'
    )

In [ ]:
sns.boxplot(
    data=df.query("education == 'Bachelor'"),
    x='salary'
    )

Nos llama la atención que en el salario de Bachelor, hay un numero notable de outliers. Investigaremos para saber a que nos puede llevar esto.

In [ ]:
sns.boxplot(
    data=df.query("education == 'Master'"),
    x='salary'
    )

In [ ]:
sns.boxplot(
    data=df.query("education == 'Doctor'"),
    x='salary'
    )

hay una clara relación entre el nivel educativo y el salario. Cuanto mayor es este, mayor es el sueldo

### ¿Cuál es la proporción de clientes con diferentes tipos de tarjetas de fidelidad?

In [ ]:
df_cont_tarjetas = df.groupby("loyalty_card")["loyalty_number"].count().reset_index()
df_cont_tarjetas

In [ ]:
plt.pie("loyalty_number", labels= "loyalty_card",
        data = df_cont_tarjetas, 
        autopct=  '%1.1f%%');

la mayor parte de las tarjetas son del tipo Star. No hay una diferencia extrema entre los tipo de tarjetas

#### NEXT STEP -- averiguar la relación entre el nivel adquisitivo y el tipo de tarjeta de fidelidad

## ¿Cómo se distribuyen los clientes según su estado civil y género?

In [ ]:
#agrupamos para ver la frecuencia de las dos columnas
df_mar_gen = df.groupby(['marital_status', 'gender']).size().reset_index(name='Customer_Count')


In [ ]:
sns.barplot(
    data = df_mar_gen,
    x='marital_status',
    y='Customer_Count',
    hue='gender');

en genral, no hay mucha diferencia entre mujeres y hombres, sin embargo, si que se obseerva que la mayoria de los clientes estan casados.

## ¿Hay alguna relación entre la ciudad del pasajero y los vuelos reservados?

In [ ]:
# 1. Agrupar por provincia y sumar el total de vuelos reservados
df_vuelos_por_provincia = df.groupby('province')['flights_booked'].sum().reset_index()

df_vuelos_por_provincia = df_vuelos_por_provincia.sort_values(by='flights_booked', ascending=False)

In [ ]:
sns.barplot(
    data=df_vuelos_por_provincia,
    x='flights_booked',
    y='province',
    order=df_vuelos_por_provincia['province']); # Asegura el orden descendente

Ontario sigue siendo la Provincia que mas vuelos reserva